Algoritmo MapReduce -- WordCount en Python
===

* *60 min* | Última modificación: Junio 22, 2019

## Definición del problema

Se desea contar la frecuencia de ocurrencia de palabras en un conjunto de documentos. Debido a los requerimientos de diseño (gran volúmen de datos y tiempos rápidos de respuesta) se desea implementar una arquitectura Big Data. Se desea implementar la solución en Python.

A continuación se generarán tres archivos de prueba para probar el sistema.

In [1]:
## Se crea el directorio de entrada
!rm -rf input output
!mkdir input

In [2]:
%%writefile input/text0.txt
Analytics is the discovery, interpretation, and communication of meaningful patterns 
in data. Especially valuable in areas rich with recorded information, analytics relies 
on the simultaneous application of statistics, computer programming and operations research 
to quantify performance.

Organizations may apply analytics to business data to describe, predict, and improve business 
performance. Specifically, areas within analytics include predictive analytics, prescriptive 
analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big 
Data Analytics, retail analytics, store assortment and stock-keeping unit optimization, 
marketing optimization and marketing mix modeling, web analytics, call analytics, speech 
analytics, sales force sizing and optimization, price and promotion modeling, predictive 
science, credit risk analysis, and fraud analytics. Since analytics can require extensive 
computation (see big data), the algorithms and software used for analytics harness the most 
current methods in computer science, statistics, and mathematics.

Writing input/text0.txt


In [3]:
%%writefile input/text1.txt
The field of data analysis. Analytics often involves studying past historical data to 
research potential trends, to analyze the effects of certain decisions or events, or to 
evaluate the performance of a given tool or scenario. The goal of analytics is to improve 
the business by gaining knowledge which can be used to make improvements or changes.

Writing input/text1.txt


In [4]:
%%writefile input/text2.txt
Data analytics (DA) is the process of examining data sets in order to draw conclusions 
about the information they contain, increasingly with the aid of specialized systems 
and software. Data analytics technologies and techniques are widely used in commercial 
industries to enable organizations to make more-informed business decisions and by 
scientists and researchers to verify or disprove scientific models, theories and 
hypotheses.

Writing input/text2.txt


## Solución

### Algoritmo Map/Reduce

![assets/map-reduce.jpg](assets/map-reduce.jpg)

### Organización de trabajos

![assets/map-reduce-jobs.jpg](assets/map-reduce-jobs.jpg)

### Prueba de la implementación fuera de Hadoop

En un sistema Hadoop, se requiere un programa para realizar el mapeo y otro para la reducción que se ejecutan independientemente; Hadoop se encarga de la coordinación entre ellos. El intercambio de información se hace a traves de texto, que es el lenguaje universal para intercambio de información. 

#### Paso 1

Se implementa la función map en Python y se guarda en el archivo `mapper.py`.

In [5]:
%%writefile mapper.py
#! /usr/bin/env python

##
## Esta es la funcion que mapea la entrada a parejas (clave, valor)
##
import sys
if __name__ == "__main__": 
    
    ##
    ## itera sobre cada linea de codigo recibida
    ## a traves del flujo de entrada
    ##
    for line in sys.stdin:
        
        ##
        ## genera las tuplas palabra \tabulador 1
        ## ya que es un conteo de palabras
        ##
        for word in line.split(): 
                   
            ##
            ## escribe al flujo estandar de salida
            ##
            sys.stdout.write("{}\t1\n".format(word))
            

Overwriting mapper.py


#### Paso 2

In [6]:
## El programa anterior se hace ejecutable
!chmod +x mapper.py 

In [7]:
## la salida de la función anterior es:
!cat ./input/text*.txt | ./mapper.py | head

Analytics	1
is	1
the	1
discovery,	1
interpretation,	1
and	1
communication	1
of	1
meaningful	1
patterns	1


#### Paso 3

Se implementa la función `reduce` y se salva en el archivo `reducer.py`.

In [8]:
%%writefile reducer.py
#!/usr/bin/env python

import sys

##
## Esta funcion reduce los elementos que 
## tienen la misma clave
##

if __name__ == '__main__': 
  
    curkey = None
    total = 0
    
    ##
    ## cada linea de texto recibida es una 
    ## entrada clave \tabulador valor
    ##
    for line in sys.stdin:
        
        key, val = line.split("\t") 
        val = int(val)
        
        if key == curkey: 
            ##
            ## No se ha cambiado de clave. Aca se 
            ## acumulan los valores para la misma clave.
            ##
            total += val  
        else:
            ##
            ## Se cambio de clave. Se reinicia el
            ## acumulador.
            ##
            if curkey is not None:
                ##
                ## una vez se han reducido todos los elementos
                ## con la misma clave se imprime el resultado en
                ## el flujo de salida
                ##
                sys.stdout.write("{}\t{}\n".format(curkey, total)) 
            
            curkey = key
            total = val

Overwriting reducer.py


#### Paso 4

In [9]:
## El archivo se hace ejecutable
!chmod +x reducer.py

#### Paso 5

Se realiza la prueba de la implementación en el directorio actual antes de realizar la ejecución en el modo pseudo-distribuido. En este caso, se simula el comportamiento de Hadoop mediante la función `sort` del sistema operativo Linux.

In [10]:
##
## La función sort hace que todos los elementos con 
## la misma clave queden en lineas consecutivas.
## Hace el papel del módulo Shuffle & Sort
##
!cat ./input/text*.txt | ./mapper.py | sort | ./reducer.py | head

a	1
about	1
aid	1
algorithms	1
analysis,	1
analysis.	1
analytics,	8
analytics.	1
analytics	8
Analytics,	1


### Mejora en la implementación 

#### Mapper

In [11]:
%%writefile mapper.py
#! /usr/bin/env python

##
## Esta es la funcion que mapea la entrada a parejas (clave, valor)
##
import sys


##
## Se usa una clase iterable para implementar el mapper.
##

class Mapper:
    
    def __init__(self, stream):
        ## 
        ## almacena el flujo de entrada como una
        ## variable del objeto
        ##
        self.stream = stream
    
    def emit(self, key, value):
        ##
        ## escribe al flujo estandar de salida
        ##
        sys.stdout.write("{}\t{}\n".format(key, value))
        
        
    def status(self, message):
        ##
        ## imprime un reporte en el flujo de error
        ## no se debe usar el stdout, ya que en este 
        ## unicamente deben ir las parejas (key, value)
        ##
        sys.stderr.write('reporter:status:{}\n'.format(message))

        
    def counter(self, counter, amount=1, group="ApplicationCounter"):
        ## 
        ## imprime el valor del contador
        ##
        sys.stderr.write('reporter:counter:{},{},{}\n'.format(group, counter, amount))
        
    def map(self):

        word_counter = 0
        
        ##
        ## imprime un mensaje a la entrada
        ##
        self.status('Iniciando procesamiento ')
            
        for word in self:
            ##
            ## cuenta la cantidad de palabras procesadas
            ##
            word_counter += 1
            
            ##
            ## por cada palabra del flujo de datos
            ## emite la pareja (word, 1)
            ##
            self.emit(key=word, value=1)

        ##
        ## imprime un mensaje a la salida
        ##
        self.counter('num_words', amount=word_counter)
        self.status('Finalizadno procesamiento ')

 
            
            
    def __iter__(self):
        ##
        ## itera sobre cada linea de codigo recibida
        ## a traves del flujo de entrada
        ##
        for line in self.stream:
            ##
            ## itera sobre cada palabra de la linea
            ## (en los ciclos for, retorna las palabras
            ## una a una)
            ##
            for word in line.split():
                ##
                ## retorna la palabra siguiente en el
                ## ciclo for
                ##
                yield word
    

if __name__ == "__main__": 
    ##
    ## inicializa el objeto con el flujo de entrada
    ##
    mapper = Mapper(sys.stdin)
    
    ##
    ## ejecuta el mapper
    ##
    mapper.map()


Overwriting mapper.py


In [12]:
## El programa anterior se hace ejecutable
!chmod +x mapper.py 

In [13]:
## la salida de la función anterior es:
!cat ./input/text*.txt | ./mapper.py | head

reporter:status:Iniciando procesamiento 
reporter:counter:ApplicationCounter,num_words,252
reporter:status:Finalizadno procesamiento 
Analytics	1
is	1
the	1
discovery,	1
interpretation,	1
and	1
communication	1
of	1
meaningful	1
patterns	1


#### Reducer

El reducer recibe las parejas (key, value) a través del flujo de salida. En los ejemplos anteriores, el reducer verifica si la clave cambia de un elemento al siguiente. Sin embargo, resulta más eficiente que se pueda iterar directamente sobre elementos consecutivos que tienen la misma clave. La función `groupby` de la librería `itertools` permite hacer esto. Dicha función recibe como argumentos los datos y una función que genera la clave para cada dato. Retorna una tupla con la clave y los elementos consecutivos que contienen la misma clave. El siguiente ejemplo permite clarificar su operación.

In [14]:
import itertools

## la letra es la clave y los números son los valores
data = [('A', 1), ('B', 10), ('A', 2), ('A', 3), ('A', 4) , ('B', 20)]

## retorna la parte correspondiente a la clave
def keyfun(x):
    k, v = x
    return k

## itera sobre la clave y los elementos que contiene 
## la misma clave
for key, group in itertools.groupby(data, keyfun):
    print(key)
    for g in group:
        print('   ', g)

A
    ('A', 1)
B
    ('B', 10)
A
    ('A', 2)
    ('A', 3)
    ('A', 4)
B
    ('B', 20)


A continuación se modifica el reducer para incoporar el uso de clases y de la función `groupby`.

In [15]:
%%writefile reducer.py
#!/usr/bin/env python

import sys
import itertools

class Reducer:
    
    def __init__(self, stream):
        self.stream = stream
        
    def emit(self, key, value):
        sys.stdout.write("{}\t{}\n".format(key, value)) 

    def reduce(self):
        ##
        ## Esta funcion reduce los elementos que 
        ## tienen la misma clave
        ##        
        for key, group in itertools.groupby(self, lambda x: x[0]):
            total = 0
            for _, val in group:
                total += val
            
            self.emit(key=key, value=total)

    def __iter__(self):
        
        for line in self.stream:
            ##
            ## Lee el stream de datos y lo parte 
            ## en (clave, valor)
            ##
            key, val = line.split("\t") 
            val = int(val)
            
            ##
            ## retorna la tupla (clave, valor)
            ## como el siguiente elemento del ciclo for
            ##
            yield (key, val)


if __name__ == '__main__': 
  
    reducer = Reducer(sys.stdin)
    reducer.reduce()


Overwriting reducer.py


In [16]:
## Se hace ejecutable el archivo
!chmod +x reducer.py

#### Ejecución

Se prueba la implementación localmente.

In [17]:
## La función sort hace que todos los elementos con 
## la misma clave queden en lineas consecutivas.
## Hace el papel del módulo Shuffle & Sort
!cat ./input/text*.txt | ./mapper.py | sort | ./reducer.py | head

reporter:status:Iniciando procesamiento 
reporter:counter:ApplicationCounter,num_words,252
reporter:status:Finalizadno procesamiento 
a	1
about	1
aid	1
algorithms	1
analysis,	1
analysis.	1
analytics,	8
analytics.	1
analytics	8
Analytics,	1


## Notas

**Combiners.--** Los combiners son *reducers* que se ejecutan sobre los resultdos que produce cada mapper antes de pasar al modulo de suffle-&-sort, con el fin de reducir la carga computacional. Suelen ser identicos a los *reducers*. Una llamada típica sería:


     $HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
        -input input \
        -output output  \
        -mapper mapper.py \
        -reducer reducer.py \
        -combiner combiner.py


**Partitioners.--** Son rutinas que controlan como se enviar las parejas (clave, valor) a cada reducers, tal que elementos con la misma clave son enviados al mismo reducer. 

**Job Chain.--** Se refiere al encadenamiento de varias tareas cuando el cómputo que se desea realizar es muy complejo para que pueda realizarse en un MapReduce.

---

In [18]:
## limpieza
!rm reducer.py mapper.py
!rm -rf input 